# Testing Heaton Database And Creating Database Objects

Initialize Django Project

In [1]:
import os, sys
import pandas as pd

def init_django(project_name=None):
    PWD = os.path.dirname(os.getcwd())
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, PWD)
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

In [2]:
init_django('heaton')

Import Models For Session App

In [3]:
from django.db import IntegrityError
from session.models import tune_type, key, tune, session, played_tune_group, played_tune, name_yer_tune

Pull in Corrected CSV data

In [4]:
os.chdir('C:\\Users\\clayt\\Documents\\Python Scripts\\CC_website_heaton_sessions\\django_heaton_example\\heaton_project\\notebooks')
heaton_corrected = pd.read_csv('heaton_master_list_corrected.csv')
heaton_corrected.drop(columns=['index'], inplace=True)
heaton_corrected.iloc[34]

session            april 11 virtual guided (slow) session
date                                           2020-04-11
tune_name                             crow's nest hop jig
tune_type                                             jig
group_id                                                9
group_order_num                                         1
start_time                                        0:42:27
end_time                                          0:45:14
key                                                   NaN
add_info                                              NaN
Name: 34, dtype: object

Inserting Data into database

In [5]:
from datetime import timedelta
import math

def duration_time(duration_string):
    """
    Take string of format Hour:Minutes:Seconds
    and return datetime.timedelta
    """

    hour, minutes, secs = duration_string.split(":")

    return timedelta(hours=float(hour),minutes=float(minutes),seconds=float(secs))

def replace_nan(s):
    """
    Replace string "nan" with ""
    """

    if isinstance(s, float):
        return ""
    if isinstance(s, str):
        return s

def heaton_database_insert(df_row):
    """
    Takes a Dataframe row and inserts into Django database
    using the imported models
    """

    # Read in Row Data
    session_row = df_row.session.lower()
    date_row = df_row.date
    tune_name_row = df_row.tune_name.lower()
    tune_type_row = df_row.tune_type.lower()
    group_id_row = df_row.group_id
    group_order_num_row = df_row.group_order_num
    start_time_row = duration_time(df_row.start_time)
    end_time_row = duration_time(df_row.end_time)
    key_row = replace_nan(df_row.key)
    add_info_row = replace_nan(df_row.add_info)

    # tune_type object lookup or create
    tune_type_obj, _ = tune_type.objects.get_or_create(tune_type_char=tune_type_row)

    # key object lookup or create
    key_obj, _ = key.objects.get_or_create(key_type_char=key_row)

    # tune object lookup or create
    tune_obj, _ = tune.objects.get_or_create(name1=tune_name_row, tune_type=tune_type_obj)

    # session object lookup or create
    session_obj, _ = session.objects.get_or_create(name=session_row, date=date_row)

    # played_tune_group object lookup or create
    played_tune_group_obj, _ = played_tune_group.objects.get_or_create(session=session_obj,
                                                                      session_order_num=group_id_row,
                                                                      start_time=start_time_row,
                                                                      end_time=end_time_row,
                                                                      offertory=False)
    
    # played_tune object lookup or create
    played_tune_obj, _ = played_tune.objects.get_or_create(tune=tune_obj,
                                                           played_tune_group=played_tune_group_obj,
                                                           key=key_obj,
                                                           group_order_num=group_order_num_row,
                                                           add_info=add_info_row)

In [6]:
def progress_in_index(n):
    """
    Print Index If Divisable by 50
    """

    if n % 50 == 0:
        print(n)

# Inserting Data into Database Models
for index, row in heaton_corrected.iterrows():
    try:
        heaton_database_insert(row)
    except Exception as e:
        print("Index {}: {}".format(index,e))
        break
    progress_in_index(index)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
